# Extract similarity scores between question and answer sentence

---

In [1]:
import numpy as np

In [162]:
ls data

glove.6B.100d.txt       glove.6B.50d.txt        train-less-than-40.xml
glove.6B.200d.txt       glove.6B.zip
glove.6B.300d.txt       glove.840B.300d.zip


In [203]:
# Map each word to an index
ndim = 100
glove_path = "./data/glove.6B.100d.txt"
with open(glove_path, "rb") as lines:
    w2idx = {line.split()[0].decode("utf-8"): i for i, line in enumerate(lines)}

In [204]:
vectors = np.empty((len(w2idx), ndim), dtype=np.float)
with open(glove_path, "rb") as lines:
    for i, line in enumerate(lines):
        vectors[i] = np.asarray(map(float, line.split()[1:]))

In [165]:
w2idx.keys()[:5]  # First few words in Word2Vec

[u'biennials', u'verplank', u'soestdijk', u'woode', u'mdbo']

---

## Extract similarity score for question answer pair

For each question, get its group of candidate answers, then compute similarity scores for each one

In [166]:
import string
from nltk.corpus import stopwords

words_to_exclude = frozenset(string.punctuation) | frozenset(["..", "..."])
words_to_exclude |= frozenset(stopwords.words("english"))

In [167]:
list(words_to_exclude)[:10]

[u'all',
 u'just',
 u'being',
 u'when',
 u'over',
 u'through',
 u'during',
 u'its',
 u'before',
 '$']

In [168]:
def get_QA_group(infile):
    question = []
    answers = []
    line = infile.readline().strip()
    if line == "":
        return None
    
    if not line.startswith("<QApairs"):
        raise Exception("Invalid data format: {}<-----".format(line))
            
    while not line.strip().startswith("</QApairs"):
        line = infile.readline().replace('\t', ' ')
        if line.strip().lower().startswith("<question"):
            line = infile.readline().replace('\t', ' ')
            question.append(line.strip())
        elif line.strip().lower().startswith("<positive"):
            line = infile.readline().replace('\t', ' ')
            answers.append(("positive", line.strip()))
        elif line.strip().lower().startswith("<negative"):
            line = infile.readline().replace('\t', ' ')
            answers.append(("negative", line.strip()))
    
    return {"question": question, "answers": answers}            
                

In [169]:
from nltk.tokenize import WhitespaceTokenizer

In [201]:
def extract_vector(sentence, exclude, w2idx, wordvectors):
    # Tokenize sentence
    splitter = WhitespaceTokenizer()
    tokens = splitter.tokenize(sentence)    
    # Remove stopwords and punctuation
    words = [t.lower() for t in tokens if (t.lower() not in exclude and t.lower() in w2idx) ]
#     print([w for w in words if w in w2idx])
#     print(words)
    
    # If we cannot find any words, we can consider returning a vector of 0
    # and set the resulting cosine similarity to 0 otherwise will result in nan
    # because cosine similarity will divide by 0.
    assert(len(words) > 0)
            
    # Average words in sentence that are in word matrix
    try:
        avg_vec = np.mean([wordvectors[w2idx[w]] for w in words if w in w2idx ] 
                                                 or [np.zeros(wordvectors.shape[1])], 
                           axis=0)
        if not np.any(avg_vec):
            print("Tokens cannot be found: {}".format(words))
        assert(np.any(avg_vec))
        return avg_vec
    except UnicodeDecodeError:
        print(line.strip())
        raise
    

In [196]:
u"the" in words_to_exclude

True

In [205]:
import scipy as sp

with open("./data/train-less-than-40.xml") as infile:
    num_questions = 0
    while infile:
        scores = []
        group = get_QA_group(infile)
        
        # Check for EOF
        if group is None:
            break
            
        question = group["question"]
        qvec = extract_vector(question[0], words_to_exclude, w2idx, vectors)

        print("Question {}".format(num_questions))
        scores_by_label = {"positive": list(), "negative": list()}
        for (label, sentence) in group["answers"]:
            vec = extract_vector(sentence, words_to_exclude, w2idx, vectors)
            # Compute similarity with qvec
            cosine_distance = sp.spatial.distance.cosine(qvec, vec)
            print("Label: {} Cosine distance: {}".format(label, cosine_distance))
            scores.append(cosine_distance)
            scores_by_label[label].append(cosine_distance)
        
        print("std: {}".format(np.std(scores)))
        print("std pos: {}".format(np.std(scores_by_label["positive"])))
        print("std neg: {}".format(np.std(scores_by_label["negative"])))        
        
        num_questions += 1

Question 0
Label: positive Cosine distance: 0.195214402347
Label: positive Cosine distance: 0.202330703095
Label: positive Cosine distance: 0.13883810479
Label: positive Cosine distance: 0.118467421872
Label: negative Cosine distance: 0.195553795948
Label: negative Cosine distance: 0.317933601625
Label: negative Cosine distance: 0.222631034828
Label: negative Cosine distance: 0.257786314301
Label: negative Cosine distance: 0.226387749827
Label: negative Cosine distance: 0.185870551432
Label: negative Cosine distance: 0.264814087906
Label: negative Cosine distance: 0.292887505113
Label: negative Cosine distance: 0.212908651604
Label: negative Cosine distance: 0.192920231288
Label: negative Cosine distance: 0.201984060402
Label: negative Cosine distance: 0.175320667381
Label: negative Cosine distance: 0.221423231866
Label: negative Cosine distance: 0.358690378091
Label: negative Cosine distance: 0.142140072996
Label: negative Cosine distance: 0.183787367612
Label: negative Cosine distanc